In [ ]:
using Random
using Statistics
using ProgressMeter
using Suppressor
using PyCall

In [ ]:
pushfirst!(pyimport("sys")."path", "");
subset_anneal = pyimport("subset_anneal")
linear_model = pyimport("sklearn.linear_model")

In [ ]:
function sim_anneal(x, y, k, score_function; seed, steps, Tmax, Tmin)
    @suppress begin
        optimizer = subset_anneal.SubsetOptimization(x, y, k, score_function, seed)
        optimizer.steps = steps
        optimizer.Tmax = Tmax
        optimizer.Tmin = Tmin
        best_idxs, _ = optimizer.anneal()
        best_idxs .+= 1
        return best_idxs
    end
end

In [ ]:
# Experiment parameters
init_seed = 1234
seed_rng = MersenneTwister(init_seed)
steps = 10_000
Tmax = 100_000
Tmin = 1
n_iter = 1_000
k = 13
n = 25
score = (x, y) -> cor(x, y)^2

In [ ]:
matches = []
ratios = []
@showprogress for i in 1:n_iter
    # Seed and generate data
    seed = rand(seed_rng, UInt128)
    data_rng = MersenneTwister(seed)
    x, y = rand(data_rng, n), rand(data_rng, n)

    # Separate programatically
    idxs_prd = sim_anneal(
        x, y, k, score; steps = steps, seed = seed, Tmax = Tmax, Tmin = Tmin)
    score_prd = score(x[idxs_prd], y[idxs_prd])

    # Separate via brute-force
    idxs_grd, _ = brute_force(x, y, k, score)
    score_grd = score(x[idxs_grd], y[idxs_grd])

    push!(ratios, score_prd / score_grd)
    push!(matches, sort(idxs_prd) == sort(idxs_grd))
end

In [ ]:
println("Average success rate: ", mean(matches))
println("Average R2 ratio: ", mean(ratios))